In [1]:
# Import Dependencies
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Read in CSVs
crimes_2019 = pd.read_csv("crimes2019.csv")
crimes_2020 = pd.read_csv("crimes2020.csv")
crimes_2021 = pd.read_csv("crimes2021.csv")
airbnb_listings = pd.read_csv("listings.csv")
area_ids = pd.read_csv("community_area_ids.csv")

In [3]:
# Concatenate crime csv files
crime_data = pd.concat([crimes_2019,crimes_2020,crimes_2021])

In [4]:
# Preview full crime data
crime_data.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,11864018,JC476123,09/24/2019 08:00:00 AM,022XX S MICHIGAN AVE,1154,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT $300 AND UNDER,COMMERCIAL / BUSINESS OFFICE,False,False,...,3.0,33.0,11,1177560.0,1889548.0,2019,10/20/2019 03:56:02 PM,41.852248,-87.623786,"(41.852248185, -87.623786256)"
1,11859805,JC471592,10/13/2019 08:30:00 PM,024XX W CHICAGO AVE,0860,THEFT,RETAIL THEFT,GROCERY FOOD STORE,False,False,...,26.0,24.0,06,1160005.0,1905256.0,2019,10/20/2019 04:03:03 PM,41.895732,-87.687784,"(41.895732399, -87.687784384)"
2,11863808,JC476236,10/05/2019 06:30:00 PM,0000X N LOOMIS ST,0810,THEFT,OVER $500,RESIDENCE,False,False,...,27.0,28.0,06,1166986.0,1900306.0,2019,10/20/2019 03:56:02 PM,41.882002,-87.662287,"(41.88200224, -87.662286977)"
3,11859727,JC471542,10/13/2019 07:00:00 PM,016XX W ADDISON ST,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,...,47.0,6.0,14,1164930.0,1923972.0,2019,10/20/2019 04:03:03 PM,41.946987,-87.669164,"(41.946987144, -87.669163602)"
4,11859656,JC471240,10/13/2019 02:10:00 PM,051XX N BROADWAY,0560,ASSAULT,SIMPLE,GAS STATION,False,False,...,47.0,3.0,08A,1167380.0,1934505.0,2019,10/20/2019 04:03:03 PM,41.975838,-87.659854,"(41.975837637, -87.659853835)"


In [5]:
crime_data.groupby("Primary Type").count()

nonviolentcrimes = ["CONCEALED CARRY LICENSE", "DECEPTIVE PRACTICE", "NARCOTICS", "PROSTITUTION", "WEAPONS VIOLATION", "STALKING", "CONCEALED CARRY LICENSE VIOLATION","CRIM SEXUAL ASSAULT", "CRIMINAL SEXUAL ASSAULT", "GAMBLING", "INTERFERENCE WITH PUBLIC OFFICER", "LIQUOR LAW VIOLATION", "NON CRIMINAL", "OBSCENITY", "OTHER OFFENSE", "OTHER NARCOTIC VIOLATION", "RITUALISM", "NON-CRIMINAL"]
crime_data_clean = crime_data.loc[~crime_data["Primary Type"].isin(nonviolentcrimes), :]
crime_data_clean['Primary Type'].value_counts()

THEFT                         111332
BATTERY                        99853
CRIMINAL DAMAGE                56944
ASSAULT                        43202
MOTOR VEHICLE THEFT            21251
BURGLARY                       19655
ROBBERY                        17651
CRIMINAL TRESPASS              11774
OFFENSE INVOLVING CHILDREN      4662
PUBLIC PEACE VIOLATION          2910
SEX OFFENSE                     2457
HOMICIDE                        1441
ARSON                           1081
INTIMIDATION                     348
KIDNAPPING                       314
PUBLIC INDECENCY                  22
HUMAN TRAFFICKING                 21
Name: Primary Type, dtype: int64

In [6]:
# Preview airbnb data
airbnb_listings.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2384,"Hyde Park - Walk to UChicago, 10 min to McCormick",2613,Rebecca,NaN,Hyde Park,41.78790,-87.58780,Private room,60,2,178,12/15/19,2.56,1,353
1,4505,394 Great Reviews. 127 y/o House. 40 yds to tr...,5775,Craig & Kathleen,NaN,South Lawndale,41.85495,-87.69696,Entire home/apt,105,2,395,7/14/20,2.81,1,155
2,7126,Tiny Studio Apartment 94 Walk Score,17928,Sarah,NaN,West Town,41.90289,-87.68182,Entire home/apt,60,2,384,3/8/20,2.81,1,321
3,9811,Barbara's Hideaway - Old Town,33004,At Home Inn,NaN,Lincoln Park,41.91769,-87.63788,Entire home/apt,65,4,49,10/23/19,0.63,9,300
4,10610,3 Comforts of Cooperative Living,2140,Lois,NaN,Hyde Park,41.79612,-87.59261,Private room,21,1,44,2/14/20,0.61,5,168


In [10]:
# Preview community area reference csv
area_ids.columns =["community_id", "community_name"]
area_ids.head()

,community_id,community_name
0,32,Loop
1,14,Albany Park
2,57,Archer Heights
3,34,Armour Square
4,70,Ashburn


In [14]:
crime_data_cleaner = crime_data_clean[['ID', 'Primary Type', 'Description', 'Location Description', 'Arrest', 'Domestic', 'Community Area', 'Year', 'Latitude', 'Longitude', 'Location']]

crime_data_cleaner.head()

,ID,Primary Type,Description,Location Description,Arrest,Domestic,Community Area,Year,Latitude,Longitude,Location
1,11859805,THEFT,RETAIL THEFT,GROCERY FOOD STORE,False,False,24.0,2019,41.895732,-87.687784,"(41.895732399, -87.687784384)"
2,11863808,THEFT,OVER $500,RESIDENCE,False,False,28.0,2019,41.882002,-87.662287,"(41.88200224, -87.662286977)"
3,11859727,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,6.0,2019,41.946987,-87.669164,"(41.946987144, -87.669163602)"
4,11859656,ASSAULT,SIMPLE,GAS STATION,False,False,3.0,2019,41.975838,-87.659854,"(41.975837637, -87.659853835)"
5,11859827,THEFT,RETAIL THEFT,APPLIANCE STORE,True,False,28.0,2019,41.877822,-87.655181,"(41.877822179, -87.655181405)"


In [19]:
#change column names 
#drop nulls
#save df into csv OR connect from jupyter notebook

crime_data_cleaner.dropna()
crime_data_cleaner.columns=["id","primary_type","description", "location_description","arrest","domestic","community_area", "year", "lat" , "lng", "location"]
crime_data_cleaner.head()

,id,primary_type,description,location_description,arrest,domestic,community_area,year,lat,lng,location
1,11859805,THEFT,RETAIL THEFT,GROCERY FOOD STORE,False,False,24.0,2019,41.895732,-87.687784,"(41.895732399, -87.687784384)"
2,11863808,THEFT,OVER $500,RESIDENCE,False,False,28.0,2019,41.882002,-87.662287,"(41.88200224, -87.662286977)"
3,11859727,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,6.0,2019,41.946987,-87.669164,"(41.946987144, -87.669163602)"
4,11859656,ASSAULT,SIMPLE,GAS STATION,False,False,3.0,2019,41.975838,-87.659854,"(41.975837637, -87.659853835)"
5,11859827,THEFT,RETAIL THEFT,APPLIANCE STORE,True,False,28.0,2019,41.877822,-87.655181,"(41.877822179, -87.655181405)"


In [20]:
crime_data_cleaner.to_csv("CLEANEST_crimedata.csv", index=False)

In [21]:
engine = create_engine("sqlite:///crime_db.sqlite")

In [22]:
crime_data_cleaner.to_sql('crimes', engine)

In [23]:
airbnb_listings.to_sql('listings',engine)

In [24]:
area_ids.to_sql('comm_areas', engine)